In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import numpy as np
import random
from torch import nn
from glob import glob
from tqdm.auto import tqdm
import pandas as pd
torch.cuda.is_available()

In [ ]:
from maatool.data.feats_itdataset_v2 import FeatsIterableDatasetV2
from maatool.models.transformer import TransformerWithSinPos

In [ ]:
import pytorch_lightning as pl 

In [ ]:
import logging
import logging.config

def configure_logging(log_level):
    handlers =  {
            "maa": {
                "class": "logging.StreamHandler",
                "formatter": "maa_basic",
                "stream": "ext://sys.stdout",
            }
    }
    CONFIG = {
        "version": 1,
        "disable_existing_loggers": False,
        "formatters": {"maa_basic": {"format": '%(asctime)s %(name)s %(pathname)s:%(lineno)d - %(levelname)s - %(message)s'}},
        "handlers": handlers,
        "loggers": {"maa": {"handlers": handlers.keys(), "level": log_level}},
        "root": {"handlers": handlers.keys(), "level": log_level}
    }
    logging.config.dictConfig(CONFIG)
configure_logging("INFO")

In [ ]:
torch.distributed.is_initialized()

In [ ]:
from collections import defaultdict

In [ ]:
def get_new_tgt(prev_tgt, hyp_logprobs, logits, topk=4):
    """
    prev_tgt - (T, N_hyp)
    hyp_logprobs - (N_hyp, )
    logits - (N_hyp, C)
    """
    assert len(prev_tgt.shape) == 2, f"{prev_tgt.shape=}"
    assert len(hyp_logprobs.shape) == 1, f"{hyp_logprobs.shape=}"
    assert len(logits.shape) == 2, f"{logits.shape=}"
    assert prev_tgt.shape[1] == hyp_logprobs.shape[0] == logits.shape[0], (
        f"{prev_tgt.shape=} {hyp_logprobs.shape=} {logits.shape=}"
    )
        
    nt_topk_logits, nt_topk_idx = logits.topk(k=topk, axis=-1)
    #print("nt_topk_idx", nt_topk_idx, nt_topk_idx.shape)
    # (N, K)
    next_tokens = nt_topk_idx.T.reshape(1, -1)
    #print("next_tokens", next_tokens, next_tokens.shape)
    # (1, N*(repeat k times)) 
    # (T, N*(repeat k times))  
    new_hyp_tgt = torch.concatenate([prev_tgt.repeat(1, topk), next_tokens], axis=0)
    #print(f"{new_hyp_tgt=}", new_hyp_tgt.shape)
    # (T+1, N*(repeat k times))
    new_scores = nt_topk_logits.T.reshape(-1)
    # N*(repeat k times)
    prew_scores = hyp_logprobs.repeat(topk)
    #print("prew_scores", prew_scores)
    # N*(repeat k times)
    new_hyp_logprob = prew_scores + new_scores
    #print("new_hyp_logprob", new_hyp_logprob)
    new_hyp_logprob, idx = new_hyp_logprob.topk(k=topk)
    #print(idx)
    new_hyps = new_hyp_tgt[:, idx]
    #print("new_hyps", new_hyps, new_hyp_logprob)
    # (T+1, N*k), (N,)
    return new_hyps, new_hyp_logprob
    

    
tgt, logits = get_new_tgt(torch.LongTensor([[1,]]), torch.tensor([-1.]), torch.tensor([[-3, -4, -7, -2, -5]]))
print(">>>>\n", tgt, logits)
get_new_tgt(tgt, logits, torch.tensor([[100,    110,  200], 
                                       [100,    110,  200], 
                                       [100,    110,  200], 
                                       [100,    110,  200]]), topk=2)

In [ ]:
def sep_ready_tgt(tgt, logprobs, eos_id=2):
    """
    tgt - (T, N)
    logprobs - (N,)
    """
    assert tgt.shape[1] == logprobs.shape[0], (
        f"{tgt.shape=} {logprobs.shape=}"
    )
    
    is_end_mask = ((tgt == eos_id).sum(axis=0) > 0)
    # (N,)
    #print(is_end_mask)
    ready_tgt = tgt[:, is_end_mask]
    ready_logprobs = logprobs[is_end_mask]
    
    ready_list = [(l.cpu().item(), t.cpu().tolist()) for l, t in zip(ready_logprobs, ready_tgt.T)]

    not_ready_tgt = tgt[:, ~is_end_mask]
    not_ready_logprobs = logprobs[~is_end_mask]
    assert not_ready_tgt.shape[1] == not_ready_logprobs.shape[0], (
        f"{not_ready_tgt.shape[1]=} {not_ready_logprobs.shape[0]=}"
    )
    return ready_list, not_ready_tgt, not_ready_logprobs


sep_ready_tgt(torch.LongTensor([[1, 1], [2, 3]]), torch.tensor([-1., -3]))

In [ ]:
def set_random_seed(seed):
    if seed < 0:
        seed = seed_from_time()
    random.seed(seed)
    np.random.seed(seed)
    torch.random.manual_seed(seed)
set_random_seed(42)

class SwipeTransformerRecognizer(pl.LightningModule):
    def __init__(self, backbone, learning_rate=1e-4, speed=42):
        super().__init__()
        self.save_hyperparameters(ignore=['backbone'])
        self.backbone = backbone
        self.ce_loss = nn.CrossEntropyLoss(ignore_index=0, reduction='mean')
        set_random_seed(speed)

    def forward(self, **kwargs):
        return self.backbone(**kwargs)
    
    def get_loss(self, batch):
        # batch - (Time, Batch, ...)
        feats = batch['feats']
        # (Time, Batch, num_feats)
        tgt = batch['targets'][:-1]
        tgt_key_padding_mask = batch['tgt_key_padding_mask'][:, 1:] 
        # (Batch, Seq-1)
        logits = self.backbone(feats=feats, 
                               tgt=tgt, 
                               src_key_padding_mask=batch['src_key_padding_mask'], 
                               tgt_key_padding_mask=tgt_key_padding_mask) 
        # (Seq-1, Batch, C)
        S, N, C = logits.shape
        targets = batch['targets'][1:]
        # (Seq-1, Batch)
        # print("loss ", logits.shape, targets.shape)
        loss = self.ce_loss(logits.view(-1, C), targets.reshape(-1))
        
        return loss

        
    def training_step(self, batch, batch_idx):
        loss = self.get_loss(batch)
        self.log('train_loss', loss, on_epoch=True, prog_bar=True,  batch_size=len(batch['uids']))
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.get_loss(batch)
        self.log('valid_loss', loss, on_step=True,  batch_size=len(batch['uids']))

    def test_step(self, batch, batch_idx):
        loss = self.get_loss(batch)
        self.log('test_loss', loss,  batch_size=len(batch['uids']))

    def configure_optimizers(self):
        # self.hparams available because we called self.save_hyperparameters()
        return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = ArgumentParser(parents=[parent_parser], add_help=False)
        parser.add_argument('--learning_rate', type=float, default=0.0001)
        return parser
    
    def predict_topk(self, dl, tokenizer, topk=4, bos_id=1, eos_id=2, max_out_len=26):
        utt2word= defaultdict(list)
        utt2logs = defaultdict(list)
        pbar = tqdm(dl)
        for batch in pbar:
            memory = self.backbone.forward_encoder(batch['feats'], 
                                              src_key_padding_mask=batch['src_key_padding_mask'])
            assert memory.shape[1] == 1, f"{memory.shape=}"
            # (SrcTime, Batch, E)
            tgt = torch.full(size=(1, 1), 
                             fill_value=bos_id, 
                             dtype=torch.long, 
                             device=memory.device)
            hyp_logprobs = torch.zeros((1), device=memory.device)
            tgt_ready = []
            mkpm = batch['src_key_padding_mask']
            for l in range(max_out_len):
                #print(f"{tgt.shape=}")
                tgt_logits = self.backbone.forward_decoder(tgt, 
                                                    memory.repeat(1, tgt.shape[1], 1), 
                                                    memory_key_padding_mask=mkpm.repeat((tgt.shape[1], 1)))
                tgt_logits = tgt_logits.log_softmax(dim=-1)
                
                new_tgt, logprobs = get_new_tgt(tgt, hyp_logprobs, tgt_logits[-1], topk=topk)
                ready, tgt, hyp_logprobs = sep_ready_tgt(new_tgt, logprobs)
                tgt_ready.extend(ready)
                if len(tgt_ready) >= topk:
                    break
                
            uid = batch['uids'][0]
            if len(tgt_ready) == 0:
                logging.warning(f"tgt_ready is 0 for {uid}. {tgt.shape=}. Use all hyps as ready hyps")
                tgt_ready = [(l.cpu().item(), t.cpu().tolist()) for l, t in zip(hyp_logprobs, tgt.T)]
                
            out_indices = []
            for logprob, indices in sorted(tgt_ready, reverse=True):
                joined = tokenizer.decode(indices) #.split()[0]
                utt2word[uid].append(joined)
                utt2logs[uid].append(logprob)
            d = '|'+'|'.join(utt2word[uid]) + "|"
            pbar.set_description(f"{d}\t".ljust(40, '='), refresh=False)
        return utt2word, utt2logs

In [ ]:
sorted([1,2,3], reverse=True)

In [ ]:
#                 nt_topk_logits, nt_topk_idx = tgt_logits.topk(k=topk, axis=-1)
#                 next_tokens = nt_topk_idx[-1:] 
#                 # (1, N, K)
#                 next_tokens = next_tokens.transpose(1, 2).reshape(1, -1)
#                 # (1, N*(repeat k times)) 
#                 new_tgt = tgt.repeat(1, topk)
#                 # (T, N*(repeat k times))  
#                 new_tgt = torch.concatenate([new_tgt, next_tokens], axis=0)
#                 #print(f"{new_tgt.shape}")
#                 # (T, N_new)
#                 new_scores = nt_topk_logits[-1].T.reshape(-1)
#                 prew_scores = tgt_logprobs.repeat(topk)
#                 tgt_logprobs = prew_scores + new_scores
                # (N,)
                
#                 is_end_mask = ((new_tgt == eos_id).sum(axis=0) > 0)
#                 # (N,)
#                 #print(is_end_mask)
#                 new_ready = new_tgt[:, is_end_mask]
#                 new_ready_logprobs = tgt_logprobs[is_end_mask]
#                 tgt_ready.extend([(l.cpu(), t.cpu()) for t, l in zip(new_ready.T, new_ready_logprobs)])
                
#                 #print(is_end)
#                 if len(tgt_ready) >= topk:
#                     break
                
#                 tgt = new_tgt[:, ~is_end_mask]
#                 tgt_logprobs = tgt_logprobs[~is_end_mask]

In [ ]:
model = TransformerWithSinPos(feats_dim=37, num_tokens=500)
pl_module = SwipeTransformerRecognizer.load_from_checkpoint('exp/models/transformer_sc/lightning_logs/version_50424998/checkpoints/last.ckpt',backbone=model, map_location='cpu' )

In [ ]:
utt2words, utt2logs = pl_module.predict_topk(val_dataloader, tokenizer=tokenizer, topk=10)

In [ ]:
utt2words

In [ ]:
accuracy(valid_ref_u2w, {k:v[0] for k, v in utt2words.items()})
# topk2 total=10000 corr=8429 err=1571, accuracy: 0.8429
# topk5 total=10000 corr=8434 err=1566, accuracy: 0.8434
# topk10 total=10000 corr=8388 err=1612, accuracy: 0.8388

In [ ]:
with open('./data/voc.txt') as f:
    vocab = frozenset(s for s in map(str.strip, f.readlines()))

In [ ]:
lv = {}
for k, v in utt2words.items():
    corr_w = None
    for w in v:
        if w in vocab:
            corr_w = w
            break
    if corr_w is None: 
        logging.warning(f"{k=} doesn't have any vocab hyp. {v=}")
        corr_w = '-'
    lv[k] = corr_w
accuracy(valid_ref_u2w, lv)
# topk10 total=10000 corr=8542 err=1458, accuracy: 0.8542


In [ ]:
test_ds =  FeatsIterableDatasetV2([f"ark:data_feats/test/feats.ark"], shuffle=False, 
                                 bos_id=1, 
                                 eos_id=2, 
                                 batch_first=False)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=1, collate_fn=test_ds.collate_pad)
#test_u2w = predict(pl_module.backbone, test_dataloader)
test_u2w, test_u2l = pl_module.predict_topk(test_dataloader, tokenizer=tokenizer, topk=8)

In [ ]:

def limit_vocab(u2w, vocab=vocab):
    lv = {}
    for k, v in u2w.items():
        corr_w = []
        for w in v:
            if w in vocab:
                corr_w.append(w)
        if len(corr_w) == 0: 
            logging.warning(f"{k=} doesn't have any vocab hyp. {v=}")
            corr_w = ['-']
        lv[k] = corr_w
    return lv
test_lv = limit_vocab(test_u2w)

In [ ]:
v, k = torch.topk(x, 2, axis=-1)
print(k)

k[1:, :, :].transpose(1,2).reshape(1, 2*3)

In [ ]:
torch.concatenate([x, k], axis=0)

In [ ]:
!head ./exp/bpe500/model.vocab

In [ ]:
val_ds = FeatsIterableDatasetV2([f"ark:data_feats/valid/feats.ark"], 
                             targets_rspecifier='ark:exp/bpe500/valid-text.int', 
                                shuffle=False,
                               bos_id=1, 
                               eos_id=2,
                               batch_first=False)
val_dataloader = torch.utils.data.DataLoader(val_ds, batch_size=1, collate_fn=val_ds.collate_pad)


In [ ]:
with open('data_feats/valid/text') as f:
    valid_ref_u2w = {u:w for u, w in   map(str.split, f.readlines())}

In [ ]:
%%time
for b in tqdm(val_dataloader):
    pass
print("Done")
print(b)

In [ ]:
print(b['feats'].shape, b['targets'].shape)

In [ ]:
train_ds = FeatsIterableDatasetV2([f"ark:{f}" for f in sorted(glob("data_feats/train/feats.*.ark"))],
                                  targets_rspecifier='ark:exp/bpe500/train-text.int.ark', 
                                  shuffle=True,
                                  bos_id=1, 
                                  eos_id=2, 
                                 batch_first=False)

#train_ds = val_ds

#
# 35799.91it/s - txt format
# vs
# 136753.6it/s - ark format

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=24, collate_fn=train_ds.collate_pad, 
                                                num_workers=8)


In [ ]:
trainer = pl.Trainer(max_epochs=6, log_every_n_steps=400, reload_dataloaders_every_n_epochs=1,
                    default_root_dir='exp/models/transformer_sc',
                    callbacks=[pl.callbacks.TQDMProgressBar(refresh_rate=100),
                              pl.callbacks.ModelCheckpoint(every_n_train_steps=20000,
                                                          save_last=True)],
                    accumulate_grad_batches=4, 
                    check_val_every_n_epoch=1)


In [ ]:
model = TransformerWithSinPos(feats_dim=37, num_tokens=500)
pl_module = SwipeTransformerRecognizer(backbone=model)

In [ ]:
with torch.no_grad():
    loss = pl_module.get_loss(b)
print(loss)

In [ ]:
for b in train_dataloader:
    pass

In [ ]:
with torch.no_grad():
    pl_module.eval()
    feats = b['feats']
    # (Batch, Time, num_feats)
    tgt = b['targets'][:, :-1]
    print(feats.shape, b['targets'].shape, tgt.shape)
    tgt_key_padding_mask = b['tgt_key_padding_mask'][:, :-1] 
    #tgt_key_padding_mask = b['tgt_key_padding_mask'][:, :-1]
    # (Batch, Seq-1)
    logits = pl_module.backbone(feats=feats, 
                           tgt=tgt, 
                           src_key_padding_mask=b['src_key_padding_mask'], 
                           tgt_key_padding_mask=tgt_key_padding_mask) 
    # (Seq-1, Batch, C)
    S, N, C = logits.shape
    targets = b['targets'][:, 1:].T
    #tgt_key_padding_mask 
    # (Seq-1, Batch)
    print("loss ", logits.shape, targets.shape)
    loss = pl_module.ce_loss(logits.view(-1, C), targets.reshape(-1))
print(loss)

In [ ]:
with torch.no_grad():
    pl_module.eval()
    l = []
    t = []
    for i in range(b['feats'].shape[0]):
        feats = b['feats'][i:i+1]
        # (Batch, Time, num_feats)
        tgt = b['targets'][i:i+1, :-1]
        print(feats.shape, b['targets'].shape, tgt.shape)
        tgt_key_padding_mask = b['tgt_key_padding_mask'][i:i+1, :-1] 
        #tgt_key_padding_mask = b['tgt_key_padding_mask'][:, :-1]
        # (Batch, Seq-1)
        logits = pl_module.backbone(feats=feats, 
                               tgt=tgt, 
                               src_key_padding_mask=b['src_key_padding_mask'][i:i+1], 
                               tgt_key_padding_mask=tgt_key_padding_mask) 
        # (Seq-1, Batch, C)
        S, N, C = logits.shape
        targets = b['targets'][i:i+1, 1:].T
        #tgt_key_padding_mask 
        # (Seq-1, Batch)
        print("loss ", logits.shape, targets.shape)
        l.append(logits.view(-1, C))
        t.append(targets.reshape(-1))
    loss = pl_module.ce_loss(torch.concatenate(l), torch.concatenate(t))
print(loss)

In [ ]:
lt, tt = torch.concatenate(l), torch.concatenate(t)
print(lt.shape, tt.shape)
lt = lt[tt!=0]
tt = tt[tt!=0]
print(lt.shape, tt.shape)
torch.nn.CrossEntropyLoss()(lt, tt)

In [ ]:
#trainer.fit(pl_module, val_dataloader, val_dataloader)

In [ ]:
trainer.fit(pl_module, train_dataloader, val_dataloader)

In [ ]:
trainer

In [ ]:
#result = trainer.test(pl_module, train_dataloader)
print(result)

In [ ]:
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
model = TransformerWithSinPos(feats_dim=37, num_tokens=500)
pl_module = SwipeTransformerRecognizer.load_from_checkpoint('exp/models/transformer_sc/lightning_logs/version_50424998/checkpoints/last.ckpt',backbone=model, map_location='cpu' )

In [ ]:
trainer = pl.Trainer(max_epochs=2, log_every_n_steps=400, reload_dataloaders_every_n_epochs=1,
                    default_root_dir='exp/models/transformer_sc',
                    callbacks=[pl.callbacks.TQDMProgressBar(refresh_rate=100),
                              pl.callbacks.ModelCheckpoint(every_n_train_steps=10000,
                                                          save_last=True)],
                    accumulate_grad_batches=4, 
                    check_val_every_n_epoch=1)

In [ ]:
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
trainer.fit(pl_module, train_dataloader, val_dataloader)

In [ ]:
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
L = torch.LongTensor([3, 4, 2, 6, 5, 1, 3, 2, 4, 7])
M = torch.arange(20) < L[:, None]
M

In [ ]:
M.shape

In [ ]:
import sentencepiece as spm
import math

In [ ]:
tokenizer = spm.SentencePieceProcessor('exp/bpe500/model.model')

In [ ]:
tokenizer.decode([[10, 11, 12], [12, 13, 15]])

In [ ]:
def predict(backbone, dl, bos_id=1, eos_id=2, max_out_len=10):

    utt2word={}
    pbar = tqdm(dl)
    for batch in pbar:
        src_embs = backbone.input_ff(batch['feats'])
        # (S, B, E)
        src_embs = backbone.positional_encoding(src_embs)
        memory = backbone.transformer.encoder(src_embs, src_key_padding_mask=batch['src_key_padding_mask'])
        # (Time, Batch, E)
        tgt = torch.full(size=(1, memory.shape[1]), fill_value=bos_id, dtype=torch.long, device=memory.device)
        for _ in range(max_out_len):
            tgt_embs = backbone.tgt_embedding(tgt) * math.sqrt(backbone.d_model)
            tgt_embs = backbone.positional_encoding(tgt_embs)
            mask = pl_module.backbone.transformer.generate_square_subsequent_mask(tgt.shape[0], device=tgt.device)
            #print(tgt_embs.shape, memory.shape)
            next_tokens = backbone.transformer.decoder(tgt_embs, memory, tgt_mask=mask)
            next_tokens = backbone.head(next_tokens).argmax(dim=-1)

            #print(tgt.shape, next_tokens.shape)
            tgt = torch.concatenate([tgt, next_tokens[-1:]], axis=0)
            #print(tgt)
            is_end = ((tgt == eos_id).sum(axis=0) > 0).all()
            #print(is_end)
            if is_end:
                break

        for uid, indices in zip(batch['uids'], tgt.T):
            out_indices = []
            for i in indices.tolist():
                out_indices.append(i)
                if i == eos_id:
                    break

            # torch.unique_consecutive(indices, dim=-1).tolist()
            #print(indices)
            joined = tokenizer.decode(out_indices).split()[0]
            pbar.set_description(f"{joined}", refresh=False)
            utt2word[uid] = joined
    return utt2word

In [ ]:
def predict_v2(backbone: TransformerWithSinPos, dl, bos_id=1, eos_id=2, max_out_len=10):

    utt2word={}
    pbar = tqdm(dl)
    for batch in pbar:
        memory = backbone.forward_encoder(batch['feats'], 
                                          src_key_padding_mask=batch['src_key_padding_mask'])
        # (SrcTime, Batch, E)
        tgt = torch.full(size=(1, memory.shape[1]), 
                         fill_value=bos_id, 
                         dtype=torch.long, 
                         device=memory.device)
        for _ in range(max_out_len):
            tgt_embs = backbone.forward_decoder(tgt, 
                                                memory, 
                                                memory_key_padding_mask=batch['src_key_padding_mask'])

            next_tokens = tgt_embs.argmax(dim=-1)

            #print(tgt.shape, next_tokens.shape)
            tgt = torch.concatenate([tgt, next_tokens[-1:]], axis=0)
            #print(tgt)
            is_end = ((tgt == eos_id).sum(axis=0) > 0).all()
            #print(is_end)
            if is_end:
                break

        for uid, indices in zip(batch['uids'], tgt.T):
            out_indices = []
            for i in indices.tolist():
                out_indices.append(i)
                if i == eos_id:
                    break

            # torch.unique_consecutive(indices, dim=-1).tolist()
            #print(indices)
            joined = tokenizer.decode(out_indices).split()[0]
            pbar.set_description(f"{joined}", refresh=False)
            utt2word[uid] = joined
    return utt2word

In [ ]:
#valid_u2w = predict_v2(pl_module.backbone, val_dataloader)
with open('data_feats/valid/text') as f:
    valid_ref_u2w = {u:w for u, w in   map(str.split, f.readlines())}
    

In [ ]:
def accuracy(ref_u2w, hyp_u2w):
    corr = 0
    err = 0
    total = len(ref_u2w)
    for u, ref in tqdm(ref_u2w.items()):
        hyp = hyp_u2w[u].strip('-')
        if ref != hyp:
            print(ref, hyp)
            err +=1
        else:
            corr +=1
    a = corr/total
    print(f"{total=} {corr=} {err=}, accuracy: {a}")
    return a
#accuracy(valid_ref_u2w, valid_u2w)
# total=10000 corr=8227 err=1773, accuracy: 0.8227


In [ ]:
test_ds =  FeatsIterableDatasetV2([f"ark:data_feats/test/feats.ark"], shuffle=False, 
                                 bos_id=1, 
                                  eos_id=2, 
                                 batch_first=False)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=1, collate_fn=test_ds.collate_pad)
test_u2w = predict(pl_module.backbone, test_dataloader)

In [ ]:
test_u2w

In [ ]:
baseline_result = pd.read_csv('exp/models/ctc_trans/lightning_logs/version_50422251/test_submit.v1.csv', sep=',', names=['main', 'second', 'third', 'trash'])
baseline_result['uid'] = [f'test-{i}' for i in range(len(baseline_result))]
baseline_result.head()

In [ ]:
baseline_result['predict'] = baseline_result.uid.apply(lambda x: test_u2w[x].strip('-'))
baseline_result.head()

In [ ]:
baseline_result[baseline_result['main']!= baseline_result['predict']]

In [ ]:
rows = []

for i, row in baseline_result.iterrows():
    old_main = row['main']
    new_main = row['predict']
    if new_main != old_main:
        new_s = old_main
        new_th = row['second']
        new_tr = row['third']
    else:
        new_s = row['second']
        new_th = row['third']
        new_tr = row['trash']
    rows.append({"main": new_main,
                "second": new_s,
                "third": new_th,
                "trash": new_tr})
        
submission = pd.DataFrame(rows)
submission.head()


In [ ]:
submission.to_csv("exp/models/transformer_sc/lightning_logs//version_50432204/test_submit.v3.csv", 
                  sep=',', header=False, index=False)

In [ ]:
baseline_result['predict'] = baseline_result.uid.apply(lambda u: test_lv[u])
baseline_result.head()

In [ ]:
rows = []

for i, row in baseline_result.iterrows():
    ps = row['predict']
    for p in [row['main'], row['second'], row['third'], row['trash']]:
        if p not in ps:
            ps.append(p)
    rows.append(ps[:4])
        
submission = pd.DataFrame(rows, columns=['main', 'second', 'third', 'trash'])
submission.head()

In [ ]:
submission.to_csv("exp/models/transformer_sc/lightning_logs/version_50424998/test_submit.v4.csv", 
                  sep=',', header=False, index=False)

In [ ]:
s5 = pd.read_csv('exp/models/transformer_sc/lightning_logs/version_50424998/test_submit.v5.csv', sep=',', names=['main', 'second', 'third', 'trash'])
s5['uid'] = [f'test-{i}' for i in range(len(s5))]
s5.head()

In [ ]:
model = TransformerWithSinPos(feats_dim=37, num_tokens=500)
pl_module = SwipeTransformerRecognizer.load_from_checkpoint('exp/models/transformer_sc/lightning_logs/version_50424998/checkpoints/last.ckpt',backbone=model, map_location='cpu' )

val_ds = FeatsIterableDatasetV2([f"ark:data_feats/valid/feats.ark"], 
                                 targets_rspecifier='ark:exp/bpe500/valid-text.int', 
                                 shuffle=False,
                                 bos_id=1, 
                                 eos_id=2,
                                 batch_first=False)
val_dataloader = torch.utils.data.DataLoader(val_ds, batch_size=1, collate_fn=val_ds.collate_pad)
utt2words, utt2logs = pl_module.predict_topk(val_dataloader, tokenizer=tokenizer, topk=10)

In [ ]:
lv = {}
for k, v in utt2words.items():
    corr_w = None
    for w in v:
        if w in vocab:
            corr_w = w
            break
    if corr_w is None: 
        logging.warning(f"{k=} doesn't have any vocab hyp. {v=}")
        corr_w = '-'
    lv[k] = corr_w
accuracy(valid_ref_u2w, lv)
# topk10 total=10000 corr=8542 err=1458, accuracy: 0.8542